In [1]:
import pandas as pd
import requests

### Leemos las 3 tablas a traves de APIs
#### Tabla Jobs

In [2]:
host = "http://localhost:5000/api"

jobs_df = pd.DataFrame.from_dict(requests.get(f"{host}/jobs").json())
jobs_df.head()

,id,job
0,0,Dummy
1,1,Marketing Assistant
2,2,VP Sales
3,3,Biostatistician IV
4,4,Account Representative II


#### Tabla Departments

In [3]:
departments_df = pd.DataFrame.from_dict(requests.get(f"{host}/departments").json())
departments_df.head()

,id,department
0,0,Dummy
1,1,Product Management
2,2,Sales
3,3,Research and Development
4,4,Business Development


#### Para la tabla de Employees, parseo la salida de la API

In [4]:
employees_df = pd.DataFrame.from_dict(requests.get(f"{host}/employees").json())
employees_df['department_id'] = pd.json_normalize(employees_df['department'])
employees_df['job_id'] = pd.json_normalize(employees_df['job'])
employees_df.drop(columns=['department', 'job'], inplace=True)
employees_df.head()

,id,name,datetime,department_id,job_id
0,1,Harold Vogt,2021-11-07T02:48:42Z,2,96
1,2,Ty Hofer,2021-05-30T05:43:46Z,8,0
2,3,Lyman Hadye,2021-09-01T23:27:38Z,5,52
3,4,Lotti Crowthe,2021-10-01T13:04:21Z,12,71
4,5,Gretna Lording,2021-10-10T22:22:17Z,6,80


### Preguntas
1. Number of employees hired for each job and department in 2021 divided by quarter. The table must be ordered alphabetically by department and job.

In [10]:
def pregunta_1(year):
    # Lógica
    # Creamos la columna del año
    nro_emp_hired_df = employees_df.copy()
    nro_emp_hired_df['datetime_dt'] = pd.to_datetime(nro_emp_hired_df['datetime'])
    nro_emp_hired_df['year'] = pd.DatetimeIndex(nro_emp_hired_df['datetime_dt']).year
    nro_emp_hired_df = nro_emp_hired_df[nro_emp_hired_df['year'] == year]
    nro_emp_hired_df['q'] = pd.PeriodIndex(nro_emp_hired_df['datetime_dt'],freq='Q').astype(str)
    nro_emp_hired_df['q'] = nro_emp_hired_df['q'].str[4:]
    nro_emp_hired_df = nro_emp_hired_df.groupby(['department_id','job_id','q'])['id'].agg('count').reset_index()
    nro_emp_hired_df.rename(columns={'id':'cant'}, inplace=True)

    # Hacemos el reporte solicitado
    preg1_1_df = nro_emp_hired_df.groupby(['department_id','job_id'], as_index=False).apply(lambda x: pd.Series({
                                                                   'Q1':x.loc[x.q=='Q1']['cant'].sum(),
                                                                   'Q2':x.loc[x.q=='Q2']['cant'].sum(),
                                                                   'Q3':x.loc[x.q=='Q3']['cant'].sum(),
                                                                   'Q4':x.loc[x.q=='Q4']['cant'].sum()}))
    
    # Mostramos los que tienen department validos (id>0)
    preg1_2_df = preg1_1_df[preg1_1_df['department_id']>0]
    preg1_3_df = preg1_2_df.merge(departments_df, how='left', left_on='department_id', right_on='id').drop(columns=['department_id','id'])
    preg1_4_df = preg1_3_df.merge(jobs_df, how='left', left_on='job_id', right_on='id').drop(columns=['job_id','id']).copy()
    preg1_4_df['year'] = year
    preg1_5_df = preg1_4_df[['year', 'department', 'job', 'Q1', 'Q2', 'Q3', 'Q4']]
    preg1_6_df = preg1_5_df.sort_values(['department', 'job'], ascending=True).reset_index().drop(columns=['index'])
    return preg1_6_df

In [11]:
# Testeamos la funcion
pregunta_1(2021).head(10)

,year,department,job,Q1,Q2,Q3,Q4
0,2021,Accounting,Account Representative IV,1,0,0,0
1,2021,Accounting,Actuary,0,1,0,0
2,2021,Accounting,Analyst Programmer,0,0,1,0
3,2021,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,2021,Accounting,Cost Accountant,0,1,0,0
5,2021,Accounting,Database Administrator III,0,0,0,1
6,2021,Accounting,Desktop Support Technician,0,0,1,0
7,2021,Accounting,Food Chemist,1,0,0,0
8,2021,Accounting,Graphic Designer,0,1,0,0
9,2021,Accounting,Health Coach III,0,0,0,1


In [12]:
# Testeamos la funcion
pregunta_1(2022).head(10)

,year,department,job,Q1,Q2,Q3,Q4
0,2022,Accounting,Assistant Manager,1,0,0,0
1,2022,Accounting,Associate Professor,1,0,0,0
2,2022,Accounting,Data Coordiator,1,0,0,0
3,2022,Accounting,Developer III,1,0,0,0
4,2022,Accounting,Executive Secretary,1,0,0,0
5,2022,Accounting,Geologist II,1,0,0,0
6,2022,Accounting,Project Manager,1,0,0,0
7,2022,Accounting,Senior Developer,1,0,0,0
8,2022,Accounting,Senior Sales Associate,1,0,0,0
9,2022,Business Development,Accountant IV,1,0,0,0


2. List of ids, name and number of employees hired of each department that hired more employees than the mean of employees hired in 2021 for all the departments, ordered by the number of employees hired (descending).

In [13]:
def pregunta_2(year):
    # Lógica
    # Creamos la columna del año
    nro_emp_hired_df = employees_df.copy()
    nro_emp_hired_df['datetime_dt'] = pd.to_datetime(nro_emp_hired_df['datetime'])
    nro_emp_hired_df['year'] = pd.DatetimeIndex(nro_emp_hired_df['datetime_dt']).year

    # Calculamos la media en 2021
    nro_emp_hired_2_df = nro_emp_hired_df.groupby(['year','department_id'])['id'].agg({'count'}).reset_index()
    nro_emp_hired_2_df = nro_emp_hired_2_df[nro_emp_hired_2_df['department_id']>0]
    nro_emp_hired_2_df = nro_emp_hired_2_df.groupby(['year'])['count'].agg({'mean'}).reset_index()
    mean_year = nro_emp_hired_2_df[nro_emp_hired_2_df['year']==year]['mean'].values[0]
    print(f"La media de {year} es {mean_year:.2f}")

    # Nos quedamos con las areas que han tenido mas que la media
    nro_emp_hired_3_df = nro_emp_hired_df[nro_emp_hired_df['department_id']>0]
    nro_emp_hired_3_df = nro_emp_hired_3_df[nro_emp_hired_3_df['year']==year]
    nro_emp_hired_3_df = nro_emp_hired_3_df.groupby(['department_id'])['id'].agg({'count'}).reset_index()
    nro_emp_hired_3_df = nro_emp_hired_3_df[nro_emp_hired_3_df['count']>=mean_year]
    nro_emp_hired_3_df = nro_emp_hired_3_df.rename(columns={'count':'hired'})
    
    nro_emp_hired_3_df = nro_emp_hired_3_df.merge(departments_df, how='left', left_on='department_id', right_on='id').drop(columns=['department_id']).copy()
    nro_emp_hired_3_df = nro_emp_hired_3_df.sort_values(['hired'], ascending=False).reset_index().drop(columns=['index'])
    nro_emp_hired_3_df = nro_emp_hired_3_df[['id','department', 'hired']]
    
    return nro_emp_hired_3_df

In [14]:
# Testeamos la funcion
pregunta_2(2021)

La media de 2021 es 139.17


,id,department,hired
0,8,Support,221
1,5,Engineering,208
2,6,Human Resources,204
3,7,Services,204
4,4,Business Development,187
5,3,Research and Development,151
6,9,Marketing,143
